In [11]:
#get first
with open('input.txt') as f:
    line = f.readline().strip()
    rules = []
    while(line.strip() != ''):
        rules.append(line)
        line = f.readline().strip()
    
    assert (f.readline().strip() == 'your ticket:')
    my_ticket = f.readline().strip()
    
    f.readline() #new line
    assert (f.readline().strip() == 'nearby tickets:')
    tickets = []
    line = f.readline().strip()
    while(line):
        tickets.append(line)
        line = f.readline().strip()

In [12]:
rules, my_ticket, tickets

(['departure location: 48-793 or 800-971',
  'departure station: 36-235 or 247-974',
  'departure platform: 25-850 or 862-964',
  'departure track: 40-173 or 193-959',
  'departure date: 42-895 or 902-955',
  'departure time: 43-692 or 715-951',
  'arrival location: 38-528 or 549-967',
  'arrival station: 43-133 or 141-963',
  'arrival platform: 40-651 or 675-951',
  'arrival track: 48-801 or 811-973',
  'class: 50-562 or 587-955',
  'duration: 43-520 or 527-968',
  'price: 44-745 or 752-950',
  'route: 41-929 or 941-963',
  'row: 37-828 or 838-958',
  'seat: 47-475 or 491-972',
  'train: 38-198 or 210-965',
  'type: 33-66 or 74-949',
  'wagon: 35-492 or 507-962',
  'zone: 35-358 or 381-965'],
 '157,59,163,149,83,131,107,89,109,113,151,53,127,97,79,103,101,173,167,61',
 ['946,293,123,156,812,202,75,884,162,420,436,60,720,719,272,642,515,724,150,875',
  '221,912,887,284,758,512,907,690,384,802,742,118,318,330,282,232,507,470,871,615',
  '810,431,649,91,53,784,300,722,904,166,210,75,491,

In [63]:
#parse rules into sub_lists then combine into single_list
rule_dict = {}
all_range = set()
for rule in rules:
    print(rule)
    name, ranges = rule.split(':')
    ranges = ranges.strip()
    rng1, _, rng2 = ranges.split(' ')
    name = name.replace(":",'').replace(' ','_')
    r1min,r1max = rng1.split('-')
    r2min,r2max = rng2.split('-')
    rule_dict[name] = {}
    rule_dict[name]['r1_min'],rule_dict[name]['r1_max'] = int(r1min),int(r1max)
    rule_dict[name]['r2_min'],rule_dict[name]['r2_max'] = int(r2min),int(r2max)
    rule_dict[name]['set'] = set([x for x in range(int(r1min), int(r1max)+1)]) | \
                              set([x for x in range(int(r2min), int(r2max)+1)])
    all_range = all_range.union(rule_dict[name]['set'])

departure location: 48-793 or 800-971
departure station: 36-235 or 247-974
departure platform: 25-850 or 862-964
departure track: 40-173 or 193-959
departure date: 42-895 or 902-955
departure time: 43-692 or 715-951
arrival location: 38-528 or 549-967
arrival station: 43-133 or 141-963
arrival platform: 40-651 or 675-951
arrival track: 48-801 or 811-973
class: 50-562 or 587-955
duration: 43-520 or 527-968
price: 44-745 or 752-950
route: 41-929 or 941-963
row: 37-828 or 838-958
seat: 47-475 or 491-972
train: 38-198 or 210-965
type: 33-66 or 74-949
wagon: 35-492 or 507-962
zone: 35-358 or 381-965


In [45]:
def check_ticket(ticket):
    nums = set([int(x) for x in ticket.split(',')])
    return nums - all_range #setdiff

In [46]:
check_ticket(my_ticket)

set()

In [47]:
all_missing = []
for ticket in tickets:
    all_missing.extend(list(check_ticket(ticket)))

In [48]:
acc = 0
for missing in all_missing:
    acc += missing
acc

32842

In [49]:
#pt 2
valid_tickets = []
for ticket in tickets:
    if len(check_ticket(ticket)) == 0:
        valid_tickets.append(ticket)

In [50]:
len(valid_tickets)

190

In [77]:
import pandas as pd
df = pd.DataFrame([ticket.split(",") for ticket in valid_tickets])
for col in df.columns:
    df[col] = df[col].astype(int)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,946,293,123,156,812,202,75,884,162,420,436,60,720,719,272,642,515,724,150,875
1,221,912,887,284,758,512,907,690,384,802,742,118,318,330,282,232,507,470,871,615
2,810,431,649,91,53,784,300,722,904,166,210,75,491,87,892,625,211,151,434,608
3,643,779,232,722,326,212,626,867,265,308,561,565,322,59,466,126,60,398,462,902
4,96,926,879,397,752,863,721,283,648,731,790,314,787,127,646,123,132,228,932,611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,82,475,337,130,491,303,633,621,770,942,274,478,682,354,758,824,232,731,886,328
186,414,616,833,141,431,118,309,332,871,397,150,868,826,215,737,907,604,627,93,422
187,304,863,471,79,522,683,508,353,908,918,516,124,418,613,382,214,422,774,65,117
188,350,643,549,409,366,649,253,885,88,417,766,221,769,331,800,683,879,683,142,812


In [118]:
df_match = pd.DataFrame(columns=rule_dict.keys(),index=range(0,len(df.columns)))
for col in df.columns:
    col_set = set(df[col].values)
    count = 0
    for key in rule_dict.keys():
        delt = col_set - rule_dict[key]['set']
        if(len(delt) == 0):
            df_match[key][col] = True
            count = count + 1
    print(col,count)

0 1
1 19
2 3
3 15
4 9
5 4
6 17
7 10
8 5
9 8
10 18
11 2
12 13
13 14
14 20
15 16
16 11
17 7
18 6
19 12


In [120]:
df_match

,departure_location,departure_station,departure_platform,departure_track,departure_date,departure_time,arrival_location,arrival_station,arrival_platform,arrival_track,class,duration,price,route,row,seat,train,type,wagon,zone
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,True,True,True,True,True,NaN,True,True,True,True,True,True,True,True,True,True,True,True
2,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
3,True,True,True,True,True,True,True,NaN,True,True,NaN,True,True,True,NaN,True,NaN,True,True,NaN
4,True,True,True,True,True,NaN,True,NaN,True,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,True,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,True,NaN,NaN
6,True,True,True,True,True,True,True,NaN,True,True,NaN,True,True,True,True,True,True,True,True,NaN
7,True,True,True,True,True,True,True,NaN,True,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,True,NaN,NaN
8,True,NaN,NaN,NaN,NaN,NaN,True,NaN,True,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,True,NaN,NaN
9,True,NaN,True,True,True,NaN,True,NaN,True,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,True,NaN,NaN


In [121]:
#try and do pairing without having to use traveling salesman...
matches = {}
removed = True

while(removed):
    removed=False
    #check columns
    for col in df_match.columns:
        if(df_match[col].count() == 1):
            val = df_match[df_match[col] == True].index.values[0]
            print(col, val)
            matches[col] = val
            df_match.drop(columns=col,inplace=True)
            df_match.drop(labels=val,inplace=True)
            removed=True
            
    #check rows
    for idx,row in df_match.iterrows():
        if(row.count() == 1):
            val = idx
            name = row.loc[row==True].index[0]
            print(name, val)
            matches[name] = val
            df_match.drop(columns=name,inplace=True)
            df_match.drop(labels=val,inplace=True)
            removed=True

arrival_station 14
class 1
zone 10
row 6
train 15
seat 3
arrival_track 13
route 12
wagon 19
duration 16
departure_time 7
departure_station 4
departure_track 9
departure_date 17


In [122]:
matches

{'arrival_station': 14,
 'class': 1,
 'zone': 10,
 'arrival_platform': 0,
 'row': 6,
 'train': 15,
 'type': 11,
 'seat': 3,
 'arrival_location': 2,
 'arrival_track': 13,
 'route': 12,
 'wagon': 19,
 'price': 5,
 'duration': 16,
 'departure_location': 8,
 'departure_time': 7,
 'departure_platform': 18,
 'departure_station': 4,
 'departure_track': 9,
 'departure_date': 17}

In [124]:
cols = [int(x) for x in my_ticket.split(',')]

In [127]:
mult = 1
for key in matches:
    if('departure' in key):
        mult *= cols[matches[key]]
mult

2628667251989